In [1]:
%load_ext aiida
%aiida

Loaded AiiDA DB environment - profile name: default.

In [3]:
from aiida import orm
from aiida_openbis.utils import bisutils
import numpy as np
from aiida.cmdline.commands.cmd_archive import create

Usage: ipykernel_launcher.py [OPTIONS] [--] OUTPUT_FILE
Try 'ipykernel_launcher.py --help' for help.

Error: No such option: --f Did you mean --force?
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1077, in main
    with self.make_context(prog_name, args, **extra) as ctx:
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 943, in make_context
    self.parse_args(ctx, args)
  File "/opt/conda/lib/python3.9/site-packages/click/core.py", line 1405, in parse_args
    opts, args, param_order = parser.parse_args(args=args)
  File "/opt/conda/lib/python3.9/site-packages/click/parser.py", line 337, in parse_args
    self._process_args_for_options(state)
  File "/opt/conda/lib/python3.9/site-packages/click/parser.py", line 364, in _process_args_for_options
    self._process_opts(arg, state)
  File "/opt/conda/lib/python3.9/site-packages/click/parser.py", line 514, in _process_opts
    self._match_long_opt(norm_long_opt, explicit_value, state)
  File "/opt/conda/lib/python3.9/site-packages/click/parser.py", line 398, in _match_long_opt
    raise NoSuchOption(opt, possibil

TypeError: object of type 'NoneType' has no len()

In [64]:
def find_first_workchain(node):
    """Find the first workchain in the provenance that created the structure node."""
    lastcalling = None
    if isinstance(node, orm.StructureData):
        previous_node = node.creator
    else:
        previous_node = node
    while previous_node is not None:
        lastcalling = previous_node
        previous_node = lastcalling.caller
    if lastcalling is not None:
        return lastcalling.label, lastcalling.pk, lastcalling.description
    return None, None, ""

In [84]:
# Connect to openBIS
session = bisutils.log_in(bisurl="openbis", bisuser="admin", bispasswd="changeit")

In [88]:
# New Simulation Experiment
project_code = "/CARBON_NANOMATERIALS/TRIANGULENE_SPIN_CHAINS"
new_experiment = session.new_experiment(project = project_code, code = "TRIANGULENE_SPIN_CHAINS_EXP_5", type = 'DEFAULT_EXPERIMENT') # How to automate the code creation?
new_experiment.props["$name"] = "Simulations"
new_experiment.save()

pk = 62
stm_workchain = load_node(pk)
structure_stm = stm_workchain.inputs.structure

geoopt_workchain_details = find_first_workchain(load_node(structure_stm.pk))
geoopt_workchain_pk = geoopt_workchain_details[1]
geoopt_workchain = load_node(geoopt_workchain_pk)

first_structure = geoopt_workchain.inputs.structure
first_structure_ase = first_structure.get_ase()
first_structure_ase.positions # Atoms positions
first_structure_ase.symbols # Atoms Symbols
first_structure_ase.pbc # PBC (X, Y, Z)
first_structure.cell # Cell vectors

collection_code = "/MATERIALS/ATOMISTIC_MODELS/ATOMISTIC_MODELS_EXP_1"
new_first_atomistic_model = session.new_sample(collection=collection_code, type='ATOMISTIC_MODEL')
new_first_atomistic_model.props['$name'] = "Atomistic Model 1"
new_first_atomistic_model.props['description'] = "A nice atomistic model"
new_first_atomistic_model.props['pbcx'] = int(first_structure_ase.pbc[0])
new_first_atomistic_model.props['pbcy'] = int(first_structure_ase.pbc[1])
new_first_atomistic_model.props['pbcz'] = int(first_structure_ase.pbc[2])
new_first_atomistic_model.props['geo-is-optimised'] = 0
new_first_atomistic_model.save()

# Geometry Optimisation
experiment_code = f"{project_code}/TRIANGULENE_SPIN_CHAINS_EXP_5"
new_simulation = session.new_sample(experiment = experiment_code, type='SIMULATION')
new_simulation.props['$name'] = "Simulation GeoOpt"
new_simulation.props['description'] = "Simulation Geometry Optimisation"
# new_simulation.props['simulation-type'] = "Something"
# new_simulation.props['wfm-connection'] = "?"
new_simulation.props['wfm-uuid'] = geoopt_workchain.uuid
new_simulation.save()

geoopt_multiplicity = geoopt_workchain.inputs.multiplicity.value
experiment_code = f"{project_code}/TRIANGULENE_SPIN_CHAINS_EXP_5"
new_geoopt_model = session.new_sample(experiment = experiment_code, type='GEO_OPTIMISATION', parents = [new_simulation, new_first_atomistic_model])
new_geoopt_model.props['$name'] = "Simulated Geometry Optimisation"
new_geoopt_model.props['multiplicity'] = geoopt_multiplicity
new_geoopt_model.save()

structure = stm_workchain.inputs.structure
structure_ase = structure.get_ase()
structure_ase.positions # Atoms positions
structure_ase.symbols # Atoms Symbols
structure_ase.pbc # PBC (X, Y, Z)
structure.cell # Cell vectors

collection_code = "/MATERIALS/ATOMISTIC_MODELS/ATOMISTIC_MODELS_EXP_1"
new_optimised_atomistic_model = session.new_sample(collection=collection_code, type='ATOMISTIC_MODEL', parents = [new_geoopt_model])
new_optimised_atomistic_model.props['$name'] = "Atomistic Model 2"
new_optimised_atomistic_model.props['description'] = "A nice atomistic model"
new_optimised_atomistic_model.props['pbcx'] = int(structure_ase.pbc[0])
new_optimised_atomistic_model.props['pbcy'] = int(structure_ase.pbc[1])
new_optimised_atomistic_model.props['pbcz'] = int(structure_ase.pbc[2])
new_optimised_atomistic_model.props['geo-is-optimised'] = 1
new_optimised_atomistic_model.save()

# Simulated Scanning Tunneling Microscopy
experiment_code = f"{project_code}/TRIANGULENE_SPIN_CHAINS_EXP_5"
new_simulation = session.new_sample(experiment = experiment_code, type = 'SIMULATION')
new_simulation.props['$name'] = "Simulation STM"
new_simulation.props['description'] = "Simulation"
# new_simulation.props['simulation-type'] = "Something"
# new_simulation.props['wfm-connection'] = "?"
new_simulation.props['wfm-uuid'] = stm_workchain.uuid
new_simulation.save()

simulated_stm_uuid = stm_workchain.uuid
structure = stm_workchain.inputs.structure
dft_params = dict(stm_workchain.inputs.dft_params)
stm_params = dict(stm_workchain.inputs.stm_params)

new_spm = session.new_sample(experiment = experiment_code, type = 'STM_SIMULATION', parents = [new_simulation, new_optimised_atomistic_model])
new_spm.props['$name'] = "Simulated STM"
new_spm.props['description'] = "A nice simulated STM"
new_spm.props['e-min'] = stm_params['--energy_range'][0]
new_spm.props['e-max'] = stm_params['--energy_range'][1]
new_spm.props['de'] = stm_params['--energy_range'][2]
new_spm.save()

experiment successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.
sample successfully created.


In [68]:
# Code that may be useful in future

# pk = 62
# workchain = load_node(pk)
# structure = workchain.inputs.structure

# all_structures = [structure]
# creator_found = True

# while creator_found:
#     creator = structure.creator
#     if creator is not None:
#         structure = creator.inputs.structure
#         all_structures.append(structure)
#     else:
#         creator_found = False